# Statistical Analysis and Cluster Analysis

## Overview
In this lecture, we will learn how to cluster geographical regions based on their attributes. We will start from simple statistics, which are correlations and regression. Then, we move on to clustering analysis and investigate the two well-known methods, K-means clustering, and hierarchical clustering. 

In python, there are numerous packages that support statistics and cluster analysis. 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import geopandas as gpd
import numpy as np
from scipy.stats import pearsonr, spearmanr
from sklearn.cluster import KMeans, AgglomerativeClustering

## Correlation and Regression: Boston housing price

Focusing on Boston, we will investigate how the variables, such as crime rate, number of rooms, etc, are correlated with the housing price. Then, examine how well their variations explain the variations in housing price, with Ordinary Least Squares (OLS) regression. 

The followings are the columns in the `boston` DataFrame. The data is obtained from http://lib.stat.cmu.edu/datasets/boston.
```python
#  CRIM     per capita crime rate by town
#  ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
#  INDUS    proportion of non-retail business acres per town
#  CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
#  NOX      nitric oxides concentration (parts per 10 million)
#  RM       average number of rooms per dwelling
#  AGE      proportion of owner-occupied units built prior to 1940
#  DIS      weighted distances to five Boston employment centres
#  RAD      index of accessibility to radial highways
#  TAX      full-value property-tax rate per \$10,000
#  PTRATIO  pupil-teacher ratio by town
#  LSTAT    % lower status of the population
#  MEDV     Median value of owner-occupied homes in $1000's
```

In [ ]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
data = pd.DataFrame(data, columns=['CRIM', 'ZN', 'INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO', 'B', 'LSTAT'])
target = raw_df.values[1::2, 2]
boston = data.merge(pd.DataFrame(target, columns=['MEDV']), left_index=True, right_index=True)
boston.drop(columns=['B'], inplace=True)

boston

### Correlation

Correlation means a statistical relationship between two variables, regardless of causality. The range of output is between -1 (negative) and +1 (positive) with 0 implying no correlation. There are several methods that reveal the correlation between variables, but in our lecture, we will investigate two major ones. 
* **Pearson's correlation** measures the linear relationship between two datasets. The calculation of the p-value relies on the assumption that each dataset is normally distributed. 
* **Spearman's correlation** does not assume that both datasets are normally distributed, unlike Pearson's correlation. It sorts the distribution of each variable, respectively, then compares their correlation. 

For the implementation in Python, we will utilize `scipy` package. The implementation of both Pearson and Spearman correlation is straightforward. They just require two inputs; distributions of two variables. For more information, visit <a href=https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html>scipy.stats.pearsonr</a> or <a href=https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html>scipy.stats.spearmanr</a>. 



In [ ]:
pearsonr_result = pearsonr(boston['CRIM'], boston['MEDV'])
print(pearsonr_result)
print(f"Pearson's r between crime rate and home median value has r {round(pearsonr_result[0],2)} and p-value {round(pearsonr_result[1],2)}")

In [ ]:
spearmanr_result = spearmanr(boston['CRIM'], boston['MEDV'])
print(spearmanr_result)
print(f"Spearman correlation between crime rate and home median value has r {round(spearmanr_result[0],2)} and p-value {round(spearmanr_result[1],2)}")

In [ ]:
independent_variables = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']
dependent_variable = ['MEDV']

for var in independent_variables:
    result = pearsonr(boston[var], boston['MEDV'])
    print(f'Correlation between {var} and home median value: r {round(result[0], 2)}, p-val {round(result[1], 2)}')

In [ ]:
sns.pairplot(boston[['CRIM', 'ZN', 'INDUS', 'RM', 'PTRATIO', 'MEDV']])
plt.show()

### Regression: Ordinary least squares (OLS)

Unlike correlation, regression requires a casualty between two variables, meaning that one variable has an impact on the other variable. Therefore, we name variables as **independent variables** and **dependent variables**. Here, we assumes that `'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT'` variables may have impacted on the housing price in Boston `MEDV`. 

For the implementation in Python, we utilize `statsmodels` package. Here, you need to be careful which variable has an impact on which variable. For more information, visit <a hef=https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html> statsmodels.regression.linear_model.OLS</a>.

In [ ]:
# Independent variables
ind = boston[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']]

# Dependent variables (Median value of owner-occupied homes in $1000's)
dep = boston[['MEDV']]

ind = sm.add_constant(ind)
model = sm.OLS(dep, ind)
results = model.fit()
print(results.summary())

The resulted summary provides extensive information, but the major thing we need to focus are `R-squared`, `Adj. R-squared` and `Prob (F-statistic)`. 

Although the entire model was significant, this implementation has some variables (`INDUS`, `AGE`) provided statistically insignificant results, so that we need to run the model one more time with two variables removed. 

In [ ]:
# Independent variables
## Insignificant varaibles (INDUS, AGE) removed
ind = boston[['CRIM', 'ZN', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT']]

# Dependent variables (Median value of owner-occupied homes in $1000's)
dep = boston[['MEDV']]

ind = sm.add_constant(ind)
model = sm.OLS(dep, ind)
results = model.fit()
print(results.summary())

Once the OLS model is constructed, we can predict housing prices in Boston (`MEDV`) based on some arbitrary inputs. The order of the input still needs to match with the original input data. For more information, please visit <a href=https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.predict.html>`statsmodels.regression.linear_model.OLS.predict`</a>.

The original order in the current model is `'CRIM', 'ZN', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'LSTAT'`.

In [ ]:
arbitrary_nums = [1,  # Constant
                  0.25651,  # CRIM
                  0,        # ZN
                  0,        # CHAS
                  0.538,    # NOX
                  6.2085,   # RM
                  3.2074,   # DIS
                  5.0,      # RAD
                  330.0,    # TAX
                  19.05,    # PTRATIO
                  11.36,    # LSTAT
                 ]

results.predict(arbitrary_nums)

In [ ]:
results.params

## Clustering

For the clustering, we will utilize the Social Vulnerability Index data of Chicago. Again, the data is made up of four different categories of vulnerability index (Socioeconomic Status, Household Composition & Disability, Minority Status & Language, and Housing Type & Transportation). With the SVI dataset, we will implement **K-means** clustering and **hierarchical** clustering and investigate how the clustering results vary based on different criteria. 

<img src="./data/svi.jpg" style="width: 600px;"/>


### K-Means Clustering
K-means clustering places the observations into a single or multi-dimensional real vector and allocates the observations into a predefined *K* number of groups based on the distance between observations. The aim of this method is to minimize the variation within clusters. Therefore, quality is indicated by the number of clusters (i.e. K) as a direct correlation. 

$$ \huge \underset{\text{S}}{\text{arg min}} \sum_{k}^{i=1}\sum_{x\in S_i}^{}\left\| x - \mu_i \right\|^2$$

For more information, please visit <a href=https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html>sklearn.cluster.KMeans</a>.

In [ ]:
from sklearn.cluster import KMeans

svi = gpd.read_file('./data/svi_chicago.shp')
svi

Let's place the variations into a two-dimensional array.

In [ ]:
svi1 = svi['RPL_THEME1']
svi2 = svi['RPL_THEME2']

fig, ax = plt.subplots(figsize=(10, 10))

ax.scatter(svi1, svi2)

Conducting K-means clustering is very straightforward. You just need to specify the number of clusters in the `KMeans()` function, and then feed the function with the array you would like to cluster. 

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(svi[['RPL_THEME1', 'RPL_THEME2']])

# `.labels_` attribute provides which cluster each observation is assigned to
kmeans.labels_

In [26]:
colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6']

svi['label'] = kmeans.labels_
svi['color'] = ''

# Assign colors based on the clustering result
for idx, row in svi.iterrows():
    if row['label'] == 0:
        svi.at[idx, 'color'] = colors[0]
    elif row['label'] == 1:
        svi.at[idx, 'color'] = colors[1]
    elif row['label'] == 2:
        svi.at[idx, 'color'] = colors[2]
    elif row['label'] == 3:
        svi.at[idx, 'color'] = colors[3]
    elif row['label'] == 4:
        svi.at[idx, 'color'] = colors[4]
        
# or 
svi['color'] = svi.apply(lambda x:colors[x['label']], axis=1)

In [ ]:
# `.cluster_centers_` provides the center of each cluster
kmeans.cluster_centers_

In [ ]:
kmeans.cluster_centers_.transpose()[0]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

# Observations that are colored based on their clustering result
ax.scatter(svi1, svi2, color=svi['color'])

# Center of each cluster
ax.scatter(kmeans.cluster_centers_.transpose()[0], 
           kmeans.cluster_centers_.transpose()[1], 
           color='black', s=100
          )

In [ ]:
svi.plot(color=svi['color'], figsize=(10,10))

How can we implement K-means clustering with 3D array? 

In [31]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(svi[['RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3']])
kmeans.labels_

In [40]:
svi['label'] = kmeans.labels_
svi['color'] = svi.apply(lambda x:colors[x['label']], axis=1)

In [ ]:
# %matplotlib notebook  
fig = plt.figure( figsize=(10,10))
ax = plt.axes(projection='3d')
# Observations that are colored based on their clustering result
ax.scatter3D(svi['RPL_THEME1'], svi['RPL_THEME2'], svi['RPL_THEME3'], color=svi['color'])

# Center of each cluster
ax.scatter3D(kmeans.cluster_centers_.transpose()[0], 
             kmeans.cluster_centers_.transpose()[1], 
             kmeans.cluster_centers_.transpose()[2], 
             color='black',
             s=50
            )
plt.show()

In [ ]:
%matplotlib inline
svi.plot(svi['color'], figsize=(10,10))

What if we consider every variable? 

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(svi[['RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4']])

svi['label'] = kmeans.labels_
svi['color'] = svi.apply(lambda x:colors[x['label']], axis=1)

svi.plot(svi['color'], figsize=(10,10))

Or, it is also possible to conduct K-means for 1-D array.

In [ ]:
kmeans.cluster_centers_.transpose()

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(svi[['RPL_THEMES']])

svi['label'] = kmeans.labels_
svi['color'] = svi.apply(lambda x:colors[x['label']], axis=1)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Observations that are colored based on their clustering result
ax[0].scatter(svi[['RPL_THEMES']], [0 for i in range(len(svi['RPL_THEMES']))], color=svi['color'])

# Center of each cluster
ax[0].scatter(kmeans.cluster_centers_.transpose()[0], 
              [0 for i in range(len(kmeans.cluster_centers_.transpose()[0]))],
              color='black', s=100
             )

svi.plot('label', ax=ax[1], color=svi['color'])

How can we determine the **optimal number of clusters**, given that the quality of K-Means clustering is indicated by the number of clusters (i.e. K) as a direct correlation?

There is a method called, **Silhouette Method**, which evaluates the current partitioning of distribution based on the equation below. 

$$\huge S = \frac{1}{N}\sum_{i}^{N}\frac{b(i)- a(i)}{\text{max}{\left\{ a(i), b(i) \right\}}}$$

where<br>
$S$: the average Silhouette coefficients of current partitioning. <br>
$N$: the number of points. <br>
$a(i)$: a cohesion indicator of a point i (i.e. distances from point i to all other points in the same cluster). <br>
$b(i)$: a separation indicator of a point i (i.e. distances from point i to all points in the other clusters). <br>
$d_0$: the threshold travel cost of the analysis. 

The Silhouette method considers both within-cluster variation and between-cluster variation, and a **higher average Silhouette coefficient (i.e. S)** means that the current clustering is well classified.

For more information, please visit <a href=https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html>`sklearn.metrics.silhouette_score`</a>. 

In [52]:
from sklearn.metrics import silhouette_score

def determine_number_of_cluster(array):
    km_silhouette = []

    for i in range(2, 11):
        KM = KMeans(n_clusters=i, n_init=100, max_iter=999)
        KM.fit(array)

        # Calculate Silhouette Scores
        preds = KM.predict(array)
        silhouette = silhouette_score(array, preds)
        km_silhouette.append(silhouette)

    return km_silhouette

In [ ]:
k_means_silhoutte = determine_number_of_cluster(svi[['RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4']])
k_means_silhoutte

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
plt.plot(range(2, 11), k_means_silhoutte, color='black', linewidth='3', marker='o', linestyle='solid')

ax.set_ylabel(f'Silhouette \n coefficient', rotation='vertical', fontsize=20)
ax.set_xlabel('Number of clusters', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.tight_layout()
plt.show()

### Hierarchical clustering

The hierarchical clustering method is initiated with every individual element (i.e. each hour in our application), aggregating observations into a higher cluster based on the distances between them (i.e. similarity). Different from the results of K-means clustering are subject to change based on K, hierarchical clustering can employ multiple linkage criteria (e.g. single, complete, or Ward’s method) to pursue the optimal results depending on the situation. 

For more information, please visit <a href=https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html>`scipy.cluster.hierarchy.linkage`</a>. 

In [56]:
from scipy.cluster import hierarchy

First linkage method to investigate is **single**, which is defined as below. 

$$d(u, v) = \text{min}\left ( dist\left (u\left [ i \right ], v\left [ j \right ] \right) \right )$$

where<br>
$i$: all points in cluster $u$ <br>
$j$: all points in cluster $v$ <br>

This linkage criteria is also known as Nearest Point Algorithm. 

In [ ]:
Z = hierarchy.linkage(svi[['RPL_THEMES']], method='single')

fig, ax = plt.subplots(figsize=(15, 5))
dn = hierarchy.dendrogram(Z=Z, ax=ax)

The next linkage method to investigate is **complete**, which is defined as below. 

$$d(u, v) = \text{max}\left ( dist\left (u\left [ i \right ], v\left [ j \right ] \right) \right )$$

where<br>
$i$: all points in cluster $u$ <br>
$j$: all points in cluster $v$ <br>

This linkage criteria is also known as the Farthest Point Algorithm or Voor Hees Algorithm.

In [ ]:
Z = hierarchy.linkage(svi[['RPL_THEMES']], method='complete')

fig, ax = plt.subplots(figsize=(15, 5))
dn = hierarchy.dendrogram(Z=Z, ax=ax)

The last linkage criteria to investigate is **Ward's method**, in which the total within-cluster variance is minimized.

In [ ]:
Z = hierarchy.linkage(svi[['RPL_THEMES']], method='ward')

fig, ax = plt.subplots(figsize=(15, 5))
dn = hierarchy.dendrogram(Z=Z, ax=ax)

In general, hierarchical clustering does not indicate the optimal number of clusters, and it rather shows how the clustering aggregates. To assign geographical locations to each cluster, you need to employ `sklearn.cluster.AgglomerativeClustering()` method. In this method, you will specify how many clusters you want to have. You can reference the resulted dendrogram to come up with the optimal number of clusters. 

In [ ]:
agg_cluster = AgglomerativeClustering(n_clusters=3, linkage='ward').fit(svi[['RPL_THEMES']])

svi['label'] = agg_cluster.labels_
svi['color'] = svi.apply(lambda x:colors[x['label']], axis=1)

svi.plot(svi['color'], figsize=(10,10))